In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rtatman/deceptive-opinion-spam-corpus")

print("Path to dataset files:", path)

100%|██████████| 456k/456k [00:00<00:00, 39.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/rtatman/deceptive-opinion-spam-corpus/versions/2


Import Dataset and Preprocess

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

data = pd.read_csv(path+"/deceptive-opinion.csv")
X = np.copy(data['text'])
y = np.copy(data['deceptive'])
y = np.array(y == 'truthful').astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


Next, build and evaluate the models:

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier

classifiers = [LinearSVC(), MultinomialNB(), KNeighborsClassifier(), DecisionTreeClassifier(max_depth=10), RandomForestClassifier(max_depth=20), AdaBoostClassifier(algorithm='SAMME', estimator=DecisionTreeClassifier(max_depth=4)), XGBClassifier()]
classifier_labels = ['Linear Regression Classifier', 'Naive Bayes', 'K-Nearest Neighbors', 'Decision Tree', 'Random Forest', 'AdaBoost', 'XGBoost']
accuracies = np.zeros(7)

index = 0
for model in classifiers:
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracies[index] = accuracy_score(y_true=y_test, y_pred=y_pred)
    print('%s Accuracy %.4f' % (classifier_labels[index], accuracies[index]))
    index += 1


Linear Regression Classifier Accuracy 0.8688
Naive Bayes Accuracy 0.8281
K-Nearest Neighbors Accuracy 0.6781
Decision Tree Accuracy 0.7156
Random Forest Accuracy 0.8406
AdaBoost Accuracy 0.7937
XGBoost Accuracy 0.7969


Train and Test Neural Network

Additional Metrics

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

Plot of